### Import Packages & Data

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import math

In [2]:
stockData = pd.read_csv('Chapter2_StockPriceData.csv')

In [3]:
stockData['Error'] = stockData.Forecasted - stockData.StockPrice
stockData

,Period,StockPrice,Forecasted,Error
0,January,35,30,-5
1,February,35,31,-4
2,March,10,30,20
3,April,5,10,5
4,May,8,12,4
5,June,10,17,7
6,July,15,18,3
7,August,20,27,7
8,September,23,29,6
9,October,21,24,3


### Explore Evaluation Metrics

In [39]:
#MSE
MSE = sk.metrics.mean_squared_error(stockData['StockPrice'], stockData['Forecasted'])
print('MSE: ', round(MSE, 1 ))

MSE:  53.7


In [40]:
#RMSE
RMSE = math.sqrt(sk.metrics.mean_squared_error(stockData['StockPrice'], stockData['Forecasted']))
print('RMSE: ', round(RMSE, 1 ))

RMSE:  7.3


In [41]:
#MAE
MAE = sk.metrics.mean_absolute_error(stockData['StockPrice'], stockData['Forecasted'])
print('MAE: ', round(MAE, 1 ))

MAE:  5.7


In [36]:
#MAPE
MAPE = sum((abs(stockData['Forecasted'] - stockData['StockPrice']))/stockData['StockPrice'])/np.count_nonzero(stockData['Period'])
print('MAPE: ', round(MAPE, 2 ))
print('MAPE Performance: ', round((1 - MAPE)*100, 2))

MAPE:  0.46
MAPE Performance:  53.53


In [42]:
#R2
R2 = sk.metrics.r2_score(stockData['StockPrice'], stockData['Forecasted'])
print('R2: ', round(R2, 1 ))

R2:  0.4


### Train-Test Split

In [6]:
y = stockData['StockPrice']

train1, test1 = sk.model_selection.train_test_split(y, test_size=0.3, shuffle=False)
forecast = train1.mean() #should be 17.25

train1 = pd.DataFrame(train1)
train1['forecast'] = forecast
train1_error = sk.metrics.mean_squared_error(train1['StockPrice'], train1['forecast'])

test1 = pd.DataFrame(test1)
test1['forecast'] = forecast
test1_error = sk.metrics.mean_squared_error(test1['StockPrice'], test1['forecast'])

print('Train Error: ', train1_error, ' vs Test Error: ', test1_error)

Train Error:  122.9375  vs Test Error:  32.4375
